In [14]:
tag2id,id2tag = {},{}  # maps tag to id, tag2id:{"VB":0 , "NNP":1}
word2id,id2word = {},{}

In [15]:
for line in open('traindata.txt'):
    items = line.split('/')
    word,tag = items[0],items[1].rstrip() #抽取每一行的单词和词性
    if word not in word2id:
        word2id[word] = len(word2id)
        id2word[len(id2word)] = word
    if tag not in tag2id:
        tag2id[tag] = len(tag2id)
        id2tag[len(id2tag)] = tag
M = len(word2id)     # M:词典大小
N = len(tag2id)      # N:词性的种类个数

In [16]:
print(M,N)

18978 54


In [17]:
print(tag2id)

{'NNP': 0, ',': 1, 'VBG': 2, 'TO': 3, 'VB': 4, 'NN': 5, 'IN': 6, 'JJ': 7, 'VBD': 8, 'NNS': 9, 'CD': 10, 'CC': 11, 'PRP': 12, 'MD': 13, 'DT': 14, '.': 15, 'VBZ': 16, 'VBN': 17, 'WDT': 18, 'VBP': 19, 'POS': 20, 'RB': 21, '$': 22, 'PRP$': 23, ':': 24, 'JJR': 25, '``': 26, "''": 27, 'WP': 28, 'JJS': 29, 'WRB': 30, 'RBR': 31, 'NNPS': 32, 'RP': 33, 'WP$': 34, 'EX': 35, '(': 36, ')': 37, 'PDT': 38, 'RBS': 39, 'FW': 40, 'UH': 41, 'SYM': 42, 'LS': 43, '#': 44, 'VBG|NN': 45, 'JJ|NN': 46, 'RB|IN': 47, 'NNS|NN': 48, 'VBN|JJ': 49, 'VB|NN': 50, 'RBR|JJR': 51, 'NN|NNS': 52, 'JJ|RB': 53}


构建 pi,A,B

In [18]:
import numpy as np
pi = np.zeros(N) # pi[i]出现在第一个位置的概率
A = np.zeros((N,M)) # A[i][j]:给定tag i，出现单词j的概率
B = np.zeros((N,N)) # B[i][j]:之前的状态是i，之后转成状态j的概率

In [36]:
def log(v):
    if v == 0:
        return np.log(v+0.000001)
    return np.log(v)

In [33]:
prev_tag = ""
for line in open('traindata.txt'):
    items = line.split('/')
    wordID,tagID = word2id[items[0]] , tag2id[items[1].rstrip()]
    if prev_tag == "": # 句子开始
        pi[tagID] +=1
        A[tagID][wordID] +=1
    else:  #不是句子开头
        A[tagID][wordID] +=1
        B[tag2id[prev_tag]][tagID] +=1
    if items[0] == ".":
        prev_tag = ""
    else:
        prev_tag = items[1].rstrip()

        
# normalize
pi = pi/sum(pi)
for i in range(N):
    A[i] /= sum(A[i])
    B[i] /= sum(B[i])
        
        
        

In [34]:
print(pi)
print(id2tag)

[1.81324111e-01 0.00000000e+00 1.00049407e-02 3.33498024e-03
 3.95256917e-03 3.68083004e-02 1.11660079e-01 3.66847826e-02
 6.17588933e-04 3.81669960e-02 8.76976285e-03 5.18774704e-02
 6.02766798e-02 2.47035573e-04 2.17267787e-01 0.00000000e+00
 1.48221344e-03 6.05237154e-03 8.64624506e-04 2.47035573e-04
 0.00000000e+00 4.73073123e-02 0.00000000e+00 7.16403162e-03
 1.72924901e-03 2.09980237e-03 7.53458498e-02 6.36116601e-02
 2.59387352e-03 1.85276680e-03 5.92885375e-03 1.97628458e-03
 2.84090909e-03 0.00000000e+00 0.00000000e+00 2.71739130e-03
 5.92885375e-03 5.92885375e-03 9.88142292e-04 3.70553360e-04
 1.23517787e-04 0.00000000e+00 0.00000000e+00 1.85276680e-03
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
{0: 'NNP', 1: ',', 2: 'VBG', 3: 'TO', 4: 'VB', 5: 'NN', 6: 'IN', 7: 'JJ', 8: 'VBD', 9: 'NNS', 10: 'CD', 11: 'CC', 12: 'PRP', 13: 'MD', 14: 'DT', 15: '.', 16: 'VBZ', 17: 'VBN',

In [35]:
print(B)

[[0.37911634 0.14189119 0.00129039 ... 0.         0.         0.        ]
 [0.13274512 0.         0.0452067  ... 0.         0.         0.        ]
 [0.03709575 0.0142676  0.00475587 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.5        ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [42]:
def viterbi(x,pi,A,B):
    """
    x：用户输入字符串或序列
    pi:initial probability of tags
    A: 给定tag，每个单词出现的概率
    B：tag之间的转移概率
    """
    x = [word2id[word]for word in x.split(" ")]  # x:[4532,334,523,335,..]
    T = len(x)
    
    dp = np.zeros((T,N)) # dp[i][j] : w1,...,wi,假设wi的tag是第j个tag
    pointer = np.array([[0 for x in range(N)] for y in range(T)])  # T*N
    
    for j in range(N): #base case 
        dp[0][j] = log(pi[j]) + log(A[j][x[0]])  #第一列
    for i in range(1,T):  #每个单词
        for j in range(N): #每个词性
            dp[i][j] = -99999
            for k in range(N): #从每一个k可以到达j
                score = dp[i-1][k] + log(B[k][j]) + log(A[j][x[i]])  #核心
                if score > dp[i][j]:
                    dp[i][j] = score
                    pointer[i][j] = k
        
    # 把最好的tag sequence找到
    best_seq = [0] *T  #best_seq=[1,3,5,23,4]                
    # step1：找出对应于最后一个单词的词性     
    best_seq[T-1] = np.argmax(dp[T-1])
    # step2:通过从后到前的循环来依次求出每个单词的词性
    for i in range(T-2,-1,-1):
        best_seq[i] = pointer[i+1][best_seq[i+1]]
    # 到目前为止，best_seq存放了对应于x的词性序列
    
    for i in range(len(best_seq)):
        print(id2tag[best_seq[i]])

In [43]:
x = "Social Security number , passport number and details about the services provided for the payment"
viterbi(x, pi, A, B)

NNP
NNP
NN
,
NN
NN
CC
NNS
IN
DT
NNS
VBN
IN
DT
NN
